In [1]:
!pip install tensorflow==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling sc

In [2]:
!python --version

Python 3.7.12


In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
! conda install -y gdown

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [5]:
!gdown --id 105A0KCAXDV69rlsy_WcB-xFb0qC0_nGr

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=105A0KCAXDV69rlsy_WcB-xFb0qC0_nGr
From (redirected): https://drive.google.com/uc?id=105A0KCAXDV69rlsy_WcB-xFb0qC0_nGr&confirm=t&uuid=80917c28-1360-4dc4-99be-be3e6c0a0f24
To: /kaggle/working/train_set.zip
100%|██████████████████████████████████████| 4.82G/4.82G [01:28<00:00, 54.3MB/s]


In [6]:
!gdown --id 105fpIsGSsnwKuiAHZp_gyjGdVwG0FlFD

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From (uriginal): https://drive.google.com/uc?id=105fpIsGSsnwKuiAHZp_gyjGdVwG0FlFD
From (redirected): https://drive.google.com/uc?id=105fpIsGSsnwKuiAHZp_gyjGdVwG0FlFD&confirm=t&uuid=c07a86f0-4d35-461e-b547-f96b51417df6
To: /kaggle/working/test_set.zip
100%|███████████████████████████████████████| 3.44G/3.44G [00:26<00:00, 132MB/s]


In [7]:
from zipfile import ZipFile
  
# loading the temp.zip and creating a zip object
with ZipFile("/kaggle/working/train_set.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path="/kaggle/working/")
    

## Training the model

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import random
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

In [9]:
im_shape = (224,224)

TRAINING_DIR = '/kaggle/working/train_set'

seed = 10
num_classes  = 6
BATCH_SIZE = 16

data_generator = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.3,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        preprocessing_function=preprocess_input,
        shear_range=0.4,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
val_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, target_size=im_shape, shuffle=True, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = val_data_generator.flow_from_directory(TRAINING_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")


Found 1144 images belonging to 6 classes.
Found 326 images belonging to 6 classes.


In [10]:
base_model=MobileNetV2(weights='imagenet',include_top=False, pooling='avg', input_shape=(im_shape[0], im_shape[1], 3)) 
base_model.trainable = False #Freeze Model

model = models.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(6,activation='softmax', kernel_initializer='random_uniform'))

# x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(1024,activation='relu')(x) 
# x=Dense(1024,activation='relu')(x)
# x=Dense(512,activation='relu')(x)
# preds=Dense(6,activation='softmax', kernel_initializer='random_uniform')(x)
    
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])               


9412608/9406464 [==============================] - 0s 0us/step


In [11]:
"""
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))
x = base_model.output
x = Flatten()(x)
x = Dense(400, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freezing pretrained layers
for layer in base_model.layers:
    layer.trainable=False
    
optimizer = Adam()
model.compile(optimizer = optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
"""

"\nbase_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(im_shape[0], im_shape[1], 3))\nx = base_model.output\nx = Flatten()(x)\nx = Dense(400, activation='relu')(x)\nx = Dropout(0.3)(x)\npredictions = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)\n\nmodel = Model(inputs=base_model.input, outputs=predictions)\n\n# Freezing pretrained layers\nfor layer in base_model.layers:\n    layer.trainable=False\n    \noptimizer = Adam()\nmodel.compile(optimizer = optimizer, loss='categorical_crossentropy',metrics=['accuracy'])\n"

In [12]:
epochs = 10

# Saving the best model
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/model.h5',
        monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,verbose=1)
]

history = model.fit(
        train_generator,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data=validation_generator,
        verbose = 1)

Epoch 1/10
72/72 [==============================] - ETA: 0s - loss: 1.8184 - accuracy: 0.1862
Epoch 00001: val_loss improved from inf to 1.79974, saving model to /kaggle/working/model.h5
72/72 [==============================] - 287s 4s/step - loss: 1.8184 - accuracy: 0.1862 - val_loss: 1.7997 - val_accuracy: 0.1902
Epoch 2/10
72/72 [==============================] - ETA: 0s - loss: 1.8041 - accuracy: 0.1949
Epoch 00002: val_loss improved from 1.79974 to 1.79606, saving model to /kaggle/working/model.h5
72/72 [==============================] - 276s 4s/step - loss: 1.8041 - accuracy: 0.1949 - val_loss: 1.7961 - val_accuracy: 0.1840
Epoch 3/10
72/72 [==============================] - ETA: 0s - loss: 1.7956 - accuracy: 0.2089
Epoch 00003: val_loss improved from 1.79606 to 1.79442, saving model to /kaggle/working/model.h5
72/72 [==============================] - 276s 4s/step - loss: 1.7956 - accuracy: 0.2089 - val_loss: 1.7944 - val_accuracy: 0.1902
Epoch 4/10
72/72 [=======================

In [13]:
from zipfile import ZipFile
  
# loading the temp.zip and creating a zip object
with ZipFile("/kaggle/working/test_set.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path="/kaggle/working/")
    

In [14]:
#test the model performance on test set
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_ds = test_gen.flow_from_directory("/kaggle/working/test_set", shuffle=False, target_size=(224,224), batch_size=16)

model = keras.models.load_model('/kaggle/working/model.h5')
model.evaluate(test_ds)

Found 800 images belonging to 6 classes.
50/50 [==============================] - 175s 4s/step - loss: 1.7614 - accuracy: 0.2163


[1.7614219188690186, 0.2162500023841858]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1280)              2257984   
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 2,916,934
Trainable params: 658,950
Non-trainable params: 2,257,984
_________________________________________________________________


In [16]:
saved_model_dir = '/kaggle/working' #means current directory

tf.saved_model.save(model, saved_model_dir) #saves to the current directory

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert() #converts our model into a .tflite model which flutter uses for ondevice machine learning

with open('model_1.tflite', 'wb') as f: #to write the converted model into a file, written as binary so add 'wb' instead of 'w'
  f.write(tflite_model)

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
                tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
                tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
            ]
tflmodel = converter.convert() 
fil = open("/kaggle/working/model_2.tflite",'wb')
fil.write(tflmodel)
fil.close()

## Single Prediction

In [18]:
'''
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

labels = {0:'ripe banana', 1: 'ripe orange', 2: 'ripe tomato', 
          3: 'unripe banana', 4: 'unripe orange', 5: 'unripe tomato' }

def predict(file_loc):
  # Load TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path="/kaggle/working/model_1.tflite")
  interpreter.allocate_tensors()

  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  img = image.load_img(file_loc, target_size=(224,224))

  img_tensor = image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor, axis=0)
  img_tensor /= 255.   

  interpreter.set_tensor(input_details[0]['index'], img_tensor)

  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  pred = np.argmax(output_data)

  if output_data[0][pred] > 0.8:
    label = labels[pred]
  else:
    label = "Can't Recognize Image, Please place well"

  return label
'''

'\nimport numpy as np\nimport tensorflow as tf\nfrom tensorflow import keras\nfrom tensorflow.keras.preprocessing import image\nfrom tensorflow.keras.applications.mobilenet_v2 import preprocess_input\n\nlabels = {0:\'ripe banana\', 1: \'ripe orange\', 2: \'ripe tomato\', \n          3: \'unripe banana\', 4: \'unripe orange\', 5: \'unripe tomato\' }\n\ndef predict(file_loc):\n  # Load TFLite model and allocate tensors.\n  interpreter = tf.lite.Interpreter(model_path="/kaggle/working/model_1.tflite")\n  interpreter.allocate_tensors()\n\n  # Get input and output tensors.\n  input_details = interpreter.get_input_details()\n  output_details = interpreter.get_output_details()\n\n  img = image.load_img(file_loc, target_size=(224,224))\n\n  img_tensor = image.img_to_array(img)\n  img_tensor = np.expand_dims(img_tensor, axis=0)\n  img_tensor /= 255.   \n\n  interpreter.set_tensor(input_details[0][\'index\'], img_tensor)\n\n  interpreter.invoke()\n\n  # The function `get_tensor()` returns a copy

In [19]:
'''
file_loc = input('Enter image Location: ')
prediction = predict(file_loc)
print('Fruit Classification: ' + prediction)
'''

"\nfile_loc = input('Enter image Location: ')\nprediction = predict(file_loc)\nprint('Fruit Classification: ' + prediction)\n"

In [20]:
'''import os
base_loc = '/kaggle/working/test_set/unripe orange/'
image_lst = os.listdir(base_loc)
for imag in image_lst:
  predict(base_loc  + imag)'''

"import os\nbase_loc = '/kaggle/working/test_set/unripe orange/'\nimage_lst = os.listdir(base_loc)\nfor imag in image_lst:\n  predict(base_loc  + imag)"